# Homework: Agents

## Preparation
First, we'll define a function that we will use when building our agent.

It will generate fake weather data:


In [1]:
import random

known_weather_data = {"berlin": 20.0}


def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Returns the weather in degrees Celsius for the given city",
    "parameters": {
        "type": "object",
        "properties": {"city": {"type": "string", "description": "Name of the city to get the weather information for"}},
        "required": ["city"],
        "additionalProperties": False,
    },
}

TODO3 is "city"

### Q2. Adding another tool

In [ ]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [ ]:
set_weather = {
    "type": "function",
    "name": "set_weather",
    "description": "Sets the weather of a city in degrees Celsius",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to set the weather information for",
            },
            "temp": {
                "type": "number",
                "description": "The temperature value in degrees Celsius, that the city should be set to",
            },
        },
        "required": ["city", "temp"],
        "additionalProperties": False,
    },
}

### Q3. Install FastMCP

In [1]:
!pip install fastmcp

  Using cached fastmcp-2.10.5-py3-none-any.whl.metadata (17 kB)
  Using cached authlib-1.6.0-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached cyclopts-3.22.2-py3-none-any.whl.metadata (11 kB)
  Using cached mcp-1.11.0-py3-none-any.whl.metadata (44 kB)
  Using cached openapi_pydantic-0.5.1-py3-none-any.whl.metadata (10 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached pyperclip-1.9.0.tar.gz (20 kB)
  Preparing metadata (setup.py) ... done
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached rich_rst-1.3.1-py3-none-any.whl.metadata (6.0 kB)
  Using cached docutils-0.21.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached sse_starlette-2.4

In [2]:
!fastmcp version

FastMCP version:                                                          2.10.5
MCP version:                                                              1.11.0
Python version:                                                          3.10.16
Platform:           Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.39
FastMCP root path: /home/wgb/miniconda3/envs/llm-zoomcamp/lib/python3.10/site-p…


### Q4. Simple MCP Server

In [4]:
!python weather_server.py



╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       STDIO                                               │
│                                                  

Let's ask the temperature in Berlin:
```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}
```

Response:
```json
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}
```


### Q6. Client

In [9]:
from fastmcp import Client
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(tools)

await main()

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Te